Najpierw uruchamiamy tą linię, żeby zainstalować pakieto do środowiska wykonawczego

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Następnie na końcu tej komórki uzupełniamy identyfikatory źródłowego i docelowego katalogu w Google Drive. Następnie klikamy play. Skrypt poprosi o autoryzacje Google Drive, po autoryzacji rozpocznie kopiowanie weryfikując czy dany plik/folder już istnieje w podanym katalogu czy nie. Skrypt obsługuje timeouty/wyjątki + rekurencyjnie sprawdza istnienie plików, żeby w razie zatrzymania i po uruchomieniu ponownie szybko przeskanował co już jest i kopiował tylko brakujące


In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import time

drive_service = build('drive', 'v3')
def copy_drive_folder_between_ids(source_folder_id, target_folder_id):
    from time import sleep

    def safe_copy(file_id, body, max_retries=5):
        for attempt in range(max_retries):
            try:
                drive_service.files().copy(fileId=file_id, body=body).execute()
                return True
            except Exception as e:
                print(f"⏳ Próba {attempt + 1} nieudana: {e}")
                sleep(2 ** attempt)
        print(f"❌ Nie udało się skopiować po {max_retries} próbach.")
        return False

    def get_existing_items(folder_id):
        existing = {}
        page_token = None
        while True:
            response = drive_service.files().list(
                q=f"'{folder_id}' in parents and trashed=false",
                spaces='drive',
                fields="nextPageToken, files(id, name, mimeType)",
                pageToken=page_token
            ).execute()
            for f in response.get('files', []):
                existing[f['name']] = f
            page_token = response.get('nextPageToken', None)
            if page_token is None:
                break
        return existing

    def copy_contents(src_id, dst_id):
        dst_existing = get_existing_items(dst_id)

        page_token = None
        while True:
            results = drive_service.files().list(
                q=f"'{src_id}' in parents and trashed=false",
                fields="nextPageToken, files(id, name, mimeType)",
                pageToken=page_token
            ).execute()

            for item in results.get('files', []):
                name = item['name']
                mime_type = item['mimeType']

                # Jeśli folder już istnieje, wchodzimy do środka i kopiujemy dalej
                if mime_type == 'application/vnd.google-apps.folder':
                    if name in dst_existing:
                        print(f"📁 Folder już istnieje: {name} → kopiuję zawartość rekurencyjnie")
                        subfolder_id = dst_existing[name]['id']
                    else:
                        folder_metadata = {
                            'name': name,
                            'mimeType': mime_type,
                            'parents': [dst_id]
                        }
                        subfolder = drive_service.files().create(body=folder_metadata, fields='id').execute()
                        subfolder_id = subfolder['id']
                        print(f"📁 Utworzono folder: {name}")

                    # Rekurencja niezależnie od tego, czy folder był nowy czy nie
                    copy_contents(item['id'], subfolder_id)

                else:
                    if name in dst_existing:
                        print(f"⏭️  Pominięto plik (już istnieje): {name}")
                        continue

                    copied_file = {
                        'name': name,
                        'parents': [dst_id]
                    }
                    if safe_copy(item['id'], copied_file):
                        print(f"✅ Skopiowano plik: {name}")
                        sleep(0.2)

            page_token = results.get('nextPageToken', None)
            if page_token is None:
                break

    # Start kopiowania
    copy_contents(source_folder_id, target_folder_id)
source_folder_id = 'SOURCE_DIRECTORY_ID'
target_folder_id = 'TARGET_DIRECTORY_ID'

copy_drive_folder_between_ids(source_folder_id, target_folder_id)